# Drugi etap projektu
Julia Jodczyk

Filip Pawłowski 
### Polecenie:
“Jakiś czas temu wprowadziliśmy konta premium, które uwalniają użytkowników od słuchania reklam. Nie są one jednak jeszcze zbyt popularne – czy możemy się dowiedzieć, które osoby są bardziej skłonne do zakupu takiego konta?”

In [1]:
import pickle
import requests
import json
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from load_data import Preprocessor, DataModel
from files_utils import randomly_split_group

## Modele

Stworzyliśmy modele klasyfikacji binarnej, które dzielą użytkowników na grupy: `kupi premium` i `nie kupi premium`. 

### Model Bazowy

Jako model bazowy, najprostszy z możliwych dla danego zadania uznajemy model naiwny, który zawsze klasyfikuje użytkowników do grupy `kupi premium`: 

In [2]:
class NaiveModel:
    def predict(self, input_df):
        user_ids = input_df.index
        mock_series = pd.Series(True, index=user_ids, name="user_id")
        return mock_series
    
base_model = NaiveModel()

with open('../microservice/saved_models/base_model.sav', 'wb') as f:
    pickle.dump(base_model, f)

### Model docelowy

Jako model docelowy, po analizie z pierwszego etapu, wybraliśmy model KNN z następującymi cechami (per użytkownik):
- miasto 
- stosunek czasu reklam do całego czasu, jaki użytkownik spędził korzystając z serwisu
- stosunkowy udział każdego typu zdarzenia (event_type) we wszystkich zdarzeniach sesji
- stosunek ilości reklam po utworach ulubionego gatunku
- ulubione gatunki użytkownika

Implementacja ekstrakcji powyższych cech została umieszczona w pliku `load_data.py`. Cechy nieliczbowe - miasto oraz ulubione gatunki zostały zakodowane sposobem one hot encoding.

In [3]:
target_model = KNeighborsClassifier()
# load data:
data_model = DataModel()
data_model.users_df = pd.read_json("../data/users.json")
df = data_model.get_merged_dfs()
preprocessed_df = Preprocessor.run(df)

In [4]:
preprocessed_df = preprocessed_df.drop_duplicates()
preprocessed_df

,user_id,premium_user,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław,Ads_ratio,...,ranchera,regional mexican,rock,rock en espanol,roots rock,singer-songwriter,soft rock,soul,tropical,vocal jazz
0,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
196,102,False,0,1,0,0,0,0,0,0.018199,...,0,0,0,0,0,1,0,0,0,0
699,103,False,0,1,0,0,0,0,0,0.011745,...,0,0,0,0,1,0,0,0,0,0
947,104,False,0,0,0,0,0,1,0,0.024229,...,0,0,0,0,0,0,0,0,1,1
1737,105,False,0,0,0,0,0,1,0,0.019117,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367214,1050,False,0,0,0,1,0,0,0,0.031597,...,0,0,0,0,0,0,0,0,0,0
367484,1051,False,0,1,0,0,0,0,0,0.024226,...,0,0,1,0,0,0,0,0,0,0
367904,1052,True,0,0,0,0,0,0,1,0.033301,...,0,0,0,0,0,1,0,0,0,0
368124,1053,False,0,1,0,0,0,0,0,0.022990,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# split data:
X, y = preprocessed_df.drop(["premium_user"], axis=1), preprocessed_df["premium_user"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=18)
target_model.fit(X_train, y_train)

with open('../microservice/saved_models/KNN_model.sav', 'wb') as f:
    pickle.dump(target_model, f)

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [6]:
base_y_hat = base_model.predict(X_test)
score = accuracy_score(y_test, base_y_hat)
score 

0.346031746031746

In [7]:
report = classification_report(y_test, base_y_hat)
print(report)

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       206
        True       0.35      1.00      0.51       109

    accuracy                           0.35       315
   macro avg       0.17      0.50      0.26       315
weighted avg       0.12      0.35      0.18       315



/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
target_y_hat = target_model.predict(X_test)
score = accuracy_score(y_test, target_y_hat)
score 

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


0.5365079365079365

In [9]:
report = classification_report(y_test, target_y_hat)
print(report)

              precision    recall  f1-score   support

       False       0.66      0.61      0.63       206
        True       0.35      0.40      0.38       109

    accuracy                           0.54       315
   macro avg       0.50      0.51      0.50       315
weighted avg       0.55      0.54      0.54       315



### Porównanie wyników offline

- Model bazowy, zawsze zwracający prawdę ma skuteczność na poziomie ok. 35% co zgadza się z rozkładem danych. Naszemu modelowi udało osiągnąć skuteczność na poziomie 53%. Jest to poprawa na poziomie 10 punktów procentowych. 
- Precyzja modelu docelowego również jest wyższa, zarówno dla klasy "kupi premium" jak i "nie kupi premium". Oznacza to, że nasz model jest lepszy w poprawnej identyfikacji klasa,
- Model docelowy ma również lepszy wynik F1-score. Podsumowując, jest on lepszy od naiwnego we wszystkich rozpatrywanych kategoriach.

Model decelowy spełnia założone kryterium sukcesu - skuteczność na poziomie wyższym niż 35%. 

### Porównanie wyników

Wyniki predykcji zbierzemy za pomocą zaimplementowanego mikroserwisu (szczegóły implementacji i API niżej). 

(Przed uruchomieniem kodu z poniższej komórki należy uruchomić mikroserwis komendą `python3 /microservice/microservice.py`) 

In [10]:
users = pd.read_json("../data/users_new.json")
users_split = users.iloc[-100:]
base_model_users, target_model_users = randomly_split_group(users_split.to_dict('records'))

In [16]:
base_url = "http://127.0.0.1:8000"

for user in target_model_users:
    requests.post(f"{base_url}/predict-with/KNN", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)
for user in base_model_users:
    requests.post(f"{base_url}/predict-with/base", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [21]:
response = requests.get(f"{base_url}/test_ab_results")
response.json()["AB_test_verdict"]

'Fail to reject H0: No significant difference in performance between A and B'

### Dalsze kroki:
- opis mikroserwisu
- analiza wyników testów